**GPX to Normalized CSV**

In [ ]:
### Librairies ###
import os
import pandas as pd
from gpx_converter import Converter




def main(input_folder, output_folder):
    
    # Lecture du dossier contenant les données Strava brutes (.gpx & .fit.gz)
    for file in os.listdir(input_folder):

        # Cas où il s'agit d'un fichier GPX
        if file.endswith('.gpx'):

            # On récupère le chemin d'accès du fichier
            file_path = os.path.join(input_folder, file)

            # Conversion en CSV et sauvegrade dans le dossier indiqué
            csv_file_from_gpx = gpx_to_csv(file_path, output_folder)

            # Formatage du fichier selon la norme souhaitée
            normalize_csv_from_gpx = gpx_csv_normalizer(csv_file_from_gpx)

            print(f"Le fichier {normalize_csv_from_gpx} a été converti et normalisé en CSV.")




def gpx_to_csv(file, output_folder):
    """
    Converti un fichier GPX en CSV

    param :
    file : fichier d'entrée à convertir 
    output_folder : dossier de stockage du fichier

    sortie : 
    fichier converti au format CSV
    """
    # Vérifier et créer le dossier de sortie si nécessaire
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Extraire le nom du fichier sans l'extension
    file_name = os.path.basename(file)
    csv_file_name = file_name.replace(".gpx", ".csv")
    output_file = os.path.join(output_folder, csv_file_name)
    
    # Conversion du fichier GPX en CSV
    Converter(input_file=file).gpx_to_csv(output_file=output_file)
   
    return output_file



def gpx_csv_normalizer(file):
    """
    Normalise un fichier CSV anciennement GPX, sous le format :
    |   nom     |    date    |   latitude    |   longitude   |

    param : 
    - file : fichier CSV non normalisé

    sortie :
    -
    """
    df = pd.read_csv(file)
    file_name = os.path.basename(file)

    data_list = []
    count = 0
    for row in df.itertuples():
        count += 1
        date, y = row.time.split(' ',1)
        data_list.append({
            "nom": file_name + str(count),
            "date": date,
            "latitude": row.latitude,
            "longitude": row.longitude
        })
        
    new_df = pd.DataFrame(data_list)
    new_df.to_csv(file, index=False, encoding="utf-8")

    return file_name



In [38]:
input_folder = "/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GPX_files"
output_folder = "/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/CSV_files"


main(input_folder, output_folder)

Le fichier 9091280542.csv a été converti et normalisé en CSV.
Le fichier 9135211199.csv a été converti et normalisé en CSV.
Le fichier 9102211132.csv a été converti et normalisé en CSV.
Le fichier 9089229915.csv a été converti et normalisé en CSV.
Le fichier 9103058222.csv a été converti et normalisé en CSV.
Le fichier 9122023334.csv a été converti et normalisé en CSV.
Le fichier 9179074237.csv a été converti et normalisé en CSV.


***

**.fit.gz to normalized CSV**

In [40]:
### Librairies ###
import os
import gzip
import shutil
import pandas as pd
from fitparse import FitFile
from gpx_converter import Converter




def main_2(input_folder, output_folder):
    
    # Lecture du dossier contenant les données Strava brutes (.gpx & .fit.gz)
    for file in os.listdir(input_folder):

        """
        
        """

        # Cas où il s'agit d'un fichier .fit.gz
        if file.endswith(".gz"):

            # On récupère le chemin d'accès du fichier
            file_path = os.path.join(input_folder, file)

            # Dézippage du fichier .gz 
            unzip_fit_file_from_gz = unzip_gz_to_fit(file_path, input_folder)
            # Conversion en CSV et sauvegarde dans le dossier indiqué
            csv_file_from_fit = fit_to_csv(unzip_fit_file_from_gz, output_folder)
            # Formatage du fichier selon la norme souhaitée
            # normalize_csv_from_fit = fit_csv_normalizer(csv_file_from_fit, output_folder)

            # print(f"Le fichier .FIT.GZ {normalize_csv_from_fit} a été converti et normalisé en CSV.")









def unzip_gz_to_fit(file, input_folder):
    """
    Dézippe un fichier .fit.gz en fichier .fit pour l'enregistrer dans son dossier d'origine

    param : 
    - file : fichier à dézipper
    - input_folder : dossier d'origine

    sortie : 
    - fichier dézippé
    """
    file_name = os.path.basename(file)  # Récupération du nom du fichier
    output_file = os.path.join(input_folder, file_name[:-3])  # Retire ".gz" de l'extension

    # Dézippe du fichier .gz passé en revue
    with gzip.open(file, 'rb') as f_in, open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print(output_file)
    return output_file



def fit_to_csv(file, output_folder):
    
    fitfile = FitFile(file)

    data_list = []

    # Extraction des enregistrements principaux dans une liste
    for record in fitfile.get_messages("record"):
        record_data = {field.name: field.value for field in record}  # Dictionnaire des données
        data_list.append(record_data)

    # Conversion
    file_name = os.path.basename(file)
    csv_file_name = file_name.replace(".fit", ".csv")
    output_file = os.path.join(output_folder, csv_file_name)

    # Création d'un dataframe et sauvegarde au format CSV
    df = pd.DataFrame(data_list)
    df.to_csv(output_file, index=False, encoding="utf-8")

    return output_file


# def fit_csv_normalizer(file, output_folder):
    
#     df = pd.read_csv(file)
#     file_name = os.path.basename(file)

#     data_list = []
#     count = 0
#     for row in df.itertuples():
#         if pd.notnull(row.position_lat) and pd.notnull(row.position_long):
#             count += 1
#             date, y = row.timestamp.split(' ',1)
#             data_list.append({
#                 "nom": file_name + str(count),
#                 "date": date,
#                 "latitude": row.position_lat,
#                 "longitude": row.position_long
#             })

#     # Stockage dans un dataframe & enregistrement au format CSV
#     new_df = pd.DataFrame(data_list)
#     new_df.to_csv(file, index=False, encoding="utf-8")  # On écrase la version du CSV non-normalisée

#     return file_name
            

In [45]:
### Librairies ###
import os
import gzip
import shutil
import pandas as pd
from fitparse import FitFile
from gpx_converter import Converter




def main_2(input_folder, output_folder):
    
    # Lecture du dossier contenant les données Strava brutes (.gpx & .fit.gz)
    for file in os.listdir(input_folder):

        """
        
        """

        # Cas où il s'agit d'un fichier .fit.gz
        if file.endswith(".gz"):

            # On récupère le chemin d'accès du fichier
            file_path = os.path.join(input_folder, file)

            # Dézippage du fichier .gz 
            unzip_fit_file_from_gz = unzip_gz_to_fit(file_path, input_folder)
            # Conversion en CSV et sauvegarde dans le dossier indiqué
            csv_file_from_fit = fit_to_csv(unzip_fit_file_from_gz, output_folder)
            # Formatage du fichier selon la norme souhaitée
            normalize_csv_from_fit = fit_csv_normalizer(csv_file_from_fit, output_folder)

            print(f"Le fichier .FIT.GZ {normalize_csv_from_fit} a été converti et normalisé en CSV.")









def unzip_gz_to_fit(file, input_folder):
    """
    Dézippe un fichier .fit.gz en fichier .fit pour l'enregistrer dans son dossier d'origine

    param : 
    - file : fichier à dézipper
    - input_folder : dossier d'origine

    sortie : 
    - fichier dézippé
    """
    file_name = os.path.basename(file)  # Récupération du nom du fichier
    output_file = os.path.join(input_folder, file_name[:-3])  # Retire ".gz" de l'extension

    # Dézippe du fichier .gz passé en revue
    with gzip.open(file, 'rb') as f_in, open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print(output_file)
    return output_file



def fit_to_csv(file, output_folder):
    
    fitfile = FitFile(file)

    data_list = []

    # Extraction des enregistrements principaux dans une liste
    for record in fitfile.get_messages("record"):
        record_data = {}
        for field in record:
            if field.name == 'position_lat' and field.value is not None:
                record_data['position_lat'] = field.value / ((2**32) / 360)
            elif field.name == 'position_long' and field.value is not None:
                record_data['position_long'] = field.value / ((2**32) / 360)
            else:
                record_data[field.name] = field.value
        data_list.append(record_data)

    # Conversion
    file_name = os.path.basename(file)
    csv_file_name = file_name.replace(".fit", ".csv")
    output_file = os.path.join(output_folder, csv_file_name)

    # Création d'un dataframe et sauvegarde au format CSV
    df = pd.DataFrame(data_list)
    df.to_csv(output_file, index=False, encoding="utf-8")

    return output_file


def fit_csv_normalizer(file, output_folder):
    
    df = pd.read_csv(file)
    file_name = os.path.basename(file)

    data_list = []
    count = 0
    for row in df.itertuples():
        if pd.notnull(row.position_lat) and pd.notnull(row.position_long):
            count += 1
            date, y = row.timestamp.split(' ',1)
            data_list.append({
                "nom": file_name + str(count),
                "date": date,
                "latitude": row.position_lat,
                "longitude": row.position_long
            })

    # Stockage dans un dataframe & enregistrement au format CSV
    new_df = pd.DataFrame(data_list)
    new_df.to_csv(file, index=False, encoding="utf-8")  # On écrase la version du CSV non-normalisée

    return file_name
            

In [47]:
input_folder = "/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GZ_files"
output_folder = "/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/CSV_files_2"

main_2(input_folder, output_folder)

/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GZ_files/10984957951.fit
Le fichier .FIT.GZ 10984957951.csv a été converti et normalisé en CSV.
/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GZ_files/10913126767.fit
Le fichier .FIT.GZ 10913126767.csv a été converti et normalisé en CSV.
/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GZ_files/10895317197.fit
Le fichier .FIT.GZ 10895317197.csv a été converti et normalisé en CSV.
/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GZ_files/10738316167.fit
Le fichier .FIT.GZ 10738316167.csv a été converti et normalisé en CSV.
/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/01_DATA_TEST/GZ_files/10937490022.fit
Le fichier .FIT.GZ 10937490022.csv a été converti et normalisé en CSV.
